In [40]:
import glob
import os
import numpy as np
from astropy.io import fits
import pickle
from copy import deepcopy

import lsst.afw.math as afwMath
import lsst.eotest.image_utils as imutils
from lsst.eotest.sensor.MaskedCCD import MaskedCCD

from scipy.ndimage.filters import gaussian_filter
import lsst.pex.config as pexConfig
import lsst.pipe.base as pipeBase
from mixcoatl.crosstalk import CrosstalkMatrix

from lsst.obs.lsst import LsstCamMapper as camMapper
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms

## For profiling
import time
import tracemalloc

## Crosstalk Code

In [112]:
def my_timer(func):
    def wrapper(*args, **kwargs):
        a = time.time()
        res = func(*args, **kwargs)
        b = time.time()
        print(func.__name__, b-a)
        return res
    return wrapper

def make_stamp(imarr, y, x, l=200):
    """Get postage stamp for crosstalk calculations."""

    maxy, maxx = imarr.shape

    y0 = max(0, y-l//2)
    y1 = min(maxy, y+l//2)

    x0 = max(0, x-l//2)
    x1 = min(maxx, x+l//2)

    return deepcopy(imarr[y0:y1, x0:x1])

def calibrated_stack(ccds, amp, gain=1.0):

    ims = [ccd.unbiased_and_trimmed_image(amp) for ccd in ccds]
    imarr = afwMath.statisticsStack(ims, afwMath.MEDIAN).getImage().getArray()*gain

    return imarr

def crosstalk_model(coefficients, aggressor_array):
    """Create a crosstalk victim postage stamp from model."""

    ny, nx = aggressor_array.shape
    crosstalk_signal = coefficients[0]
    bias = coefficients[1]
    tilty = coefficients[2]
    tiltx = coefficients[3]

    Y, X = np.mgrid[:ny, :nx]
    model = crosstalk_signal*aggressor_array + tilty*Y + tiltx*X + bias
    return model

## Maybe speed up without masked arrays
@my_timer
def crosstalk_model_fit(aggressor_stamp, victim_stamp, num_iter=3, nsig=5.0, noise=7.0):
    """Perform a crosstalk model fit for given  aggressor and victim stamps."""

    coefficients = np.asarray([[0,0,0,0]])
    victim_array = np.ma.masked_invalid(victim_stamp)
    mask = np.ma.getmask(victim_array)

    for i in range(num_iter):
        #
        # Mask outliers using residual
        #
        model = np.ma.masked_where(mask, crosstalk_model(coefficients[0],
                                                         aggressor_stamp))
        residual = victim_array - model
        res_mean = residual.mean()
        res_std = residual.std()
        victim_array = np.ma.masked_where(np.abs(residual-res_mean) \
                                              > nsig*res_std, victim_stamp)
        mask = np.ma.getmask(victim_array)
        #
        # Construct masked, compressed basis arrays
        #
        ay, ax = aggressor_stamp.shape
        bias = np.ma.masked_where(mask, np.ones((ay, ax))).compressed()
        Y, X = np.mgrid[:ay, :ax]
        Y = np.ma.masked_where(mask, Y).compressed()
        X = np.ma.masked_where(mask, X).compressed()
        aggressor_array = np.ma.masked_where(mask, aggressor_stamp).compressed()
        #
        # Perform least-squares minimization
        #
        b = victim_array.compressed()/noise
        A = np.vstack([aggressor_array, bias, Y, X]).T/noise
        coeffs, res, rank, s = np.linalg.lstsq(A, b, rcond=-1)
        covar = np.linalg.inv(np.dot(A.T, A))
        dof = b.shape[0] - 4
        
    return np.concatenate((coeffs, np.sqrt(covar.diagonal()), res, [dof]))

class CrosstalkConfig(pexConfig.Config):
    
    nsig = pexConfig.Field("Outlier rejection sigma threshold", float, 
                           default=5.0)
    num_iter = pexConfig.Field("Number of least square iterations", int, 
                               default=3)
    threshold = pexConfig.Field("Aggressor spot mean signal threshold", float,
                                default=40000.)
    output_file = pexConfig.Field("Output filename", str, 
                                  default='crosstalk_matrix.fits')
    verbose = pexConfig.Field("Turn verbosity on", bool, default=True)

class CrosstalkTask(pipeBase.Task):

    ConfigClass = CrosstalkConfig
    _DefaultName = "CrosstalkTask"

    @pipeBase.timeMethod
    def run(self, sensor_id1, infiles1, gains1, bias_frame1=None, dark_frame1=None,
            crosstalk_matrix_file=None, **kwargs):
        
        a = time.time()

        ## Parse kwargs for separate victim parameters
        try:
            sensor_id2 = kwargs['sensor_id2']
        except KeyError:
            sensor_id2 = sensor_id1
            infiles2 = infiles1
            gains2 = gains1
            bias_frame2 = bias_frame1
            dark_frame2 = dark_frame1
        else:
            infiles2 = kwargs['infiles2']
            gains2 = kwargs['gains2']
            bias_frame2 = kwargs['bias_frame2']
            dark_frame2 = kwargs['dark_frame2']
                    
        ccd1 = MaskedCCD(infiles1[0], bias_frame=bias_frame1, 
                           dark_frame=dark_frame1)
        ccd2 = MaskedCCD(infiles2[0], bias_frame=bias_frame2,
                           dark_frame=dark_frame2)
        
        all_amps = imutils.allAmps(infiles1[0])

        ## Create new matrix or modify existing
        if crosstalk_matrix_file is not None:
            crosstalk_matrix = CrosstalkMatrix(sensor_id1, sensor_id2, 
                                               crosstalk_matrix_file, 
                                               namps=len(all_amps))
            output_file = crosstalk_matrix_file
        else:
            output_file = os.path.join(self.config.output_file)
            crosstalk_matrix = CrosstalkMatrix(sensor_id1, sensor_id2, 
                                               namps=len(all_amps))

        num_aggressors = 0

        ## Search each amp for aggressor spot
        for amp in all_amps:
            print('Start Aggressor', amp, time.time()-a)
#            imarr1 = calibrated_stack(ccds1, amp, gain=gains1[amp])
            imarr1 = ccd1.unbiased_and_trimmed_image(amp).getImage().getArray()*gains1[amp]
            smoothed = gaussian_filter(imarr1, 20)
            y, x = np.unravel_index(smoothed.argmax(), smoothed.shape)
            stamp1 = make_stamp(imarr1, y, x)
            ly, lx = stamp1.shape
            Y, X = np.ogrid[-ly/2:ly/2, -lx/2:lx/2]
            mask = X*X + Y*Y <= 20*20
            if np.mean(stamp1[mask]) > self.config.threshold:
                print('Tested aggressor spot', time.time()-a)

                row = {}

                ## Calculate crosstalk for each victim amp
                for i in all_amps:
#                    imarr2 = calibrated_stack(ccds2, i, gain=gains2[i])
                    imarr2 = ccd2.unbiased_and_trimmed_image(i).getImage().getArray()*gains2[i]

                    stamp2 = make_stamp(imarr2, y, x)
                    row[i] = crosstalk_model_fit(stamp1, stamp2, noise=7.0,
                                                 num_iter=3,
                                                 nsig=self.config.nsig)
                    print('Calculated crosstalk {0}'.format(i), time.time()-a)

                crosstalk_matrix.set_row(amp, row)
                if num_aggressors == 4: break

#        crosstalk_matrix.write_fits(output_file)

In [113]:
## Script command line arguments
sensor_id = 'R21_S11'
calib_dir= '/nfs/slac/g/ki/ki19/lsst/snyder18/LSST/Data/BOT/6813D_xtalk/calibration/'
output_dir = './'
main_dir = '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_archive-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6813D/BOT_acq/v0/47875/'

a = time.time()
camera = camMapper._makeCamera()
lct = LsstCameraTransforms(camera)
gain_results = pickle.load(open(os.path.join(calib_dir, 
                                             'et_results.pkl'), 'rb'))
subdir_list = [x.path for x in os.scandir(main_dir) if os.path.isdir(x.path)]

bias_frame = os.path.join(calib_dir, '{0}_superbias.fits'.format(sensor_id))
dark_frame = os.path.join(calib_dir, '{0}_superdark.fits'.format(sensor_id))
gains = gain_results.get_amp_gains(sensor_id)

position_set = set()
for subdir in subdir_list:
    basename = os.path.basename(subdir)
    if "xtalk" not in basename:
        continue
    xpos, ypos = basename.split('_')[-4:-2]
    central_sensor, ccdX, ccdY = lct.focalMmToCcdPixel(float(ypos), float(xpos))
    if central_sensor == sensor_id:
        position_set.add((xpos, ypos))

## Only do one for now
#for i, pos in enumerate(position_set):
i = 0
pos = list(position_set)[0]
xpos, ypos = pos
infiles = glob.glob(os.path.join(main_dir, '*{0}_{1}*'.format(xpos, ypos),
                                 '*{0}.fits'.format(sensor_id)))

#    if i == 0:
print('Start Task:', time.time()-a)

output_file = os.path.join(output_dir, 
                           '{0}_{0}_crosstalk_matrix.fits'.format(sensor_id))
crosstalk_task = CrosstalkTask()
crosstalk_task.config.output_file = output_file
crosstalk_task.run(sensor_id, infiles, gains, bias_frame=bias_frame,
                   dark_frame=dark_frame)

print("Total Runtime:", time.time()-a)

Start Task: 5.175990581512451
Start Aggressor 1 1.3999738693237305
Start Aggressor 2 1.9174001216888428
Start Aggressor 3 2.3091185092926025
Start Aggressor 4 2.647634506225586
Start Aggressor 5 2.9508984088897705
Start Aggressor 6 3.254344940185547
Start Aggressor 7 3.5575029850006104
Start Aggressor 8 3.862314224243164
Start Aggressor 9 4.1673290729522705
Start Aggressor 10 4.437700986862183
Start Aggressor 11 4.701313495635986
Start Aggressor 12 4.9535071849823
Start Aggressor 13 5.1986072063446045
Tested aggressor spot 5.4361865520477295
crosstalk_model_fit 0.016791820526123047
Calculated crosstalk 1 5.522500514984131
crosstalk_model_fit 0.017472505569458008
Calculated crosstalk 2 5.607625246047974
crosstalk_model_fit 0.022731304168701172
Calculated crosstalk 3 5.700538158416748
crosstalk_model_fit 0.016728639602661133
Calculated crosstalk 4 5.7843592166900635
crosstalk_model_fit 0.022155284881591797
Calculated crosstalk 5 5.8738484382629395
crosstalk_model_fit 0.01653432846069336


In [80]:
12*4/60.

0.8